# LLM Queries in DuckDB

This notebook walks through how to call LLMs directly as a UDF in a DuckDB database using [vLLM](https://github.com/vllm-project/vllm) as the inference engine.

## Initialize the LLM Engine

In [1]:
import llmsql
from llmsql.llm.vllm import vLLM
from vllm import EngineArgs

args = EngineArgs(model="TheBloke/Llama-2-13B-chat-GPTQ")

# Initialize llmsql
llmsql.init(vLLM(engine_args=args))


Starting vLLM engine...
WARNING 04-10 18:22:10 config.py:211] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 04-10 18:22:10 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='TheBloke/Llama-2-13B-chat-GPTQ', tokenizer='TheBloke/Llama-2-13B-chat-GPTQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
INFO 04-10 18:22:11 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 04-10 18:22:11 selector.py:25] Using XFormers backend.
INFO 04-10 18:22:14 weight_utils.py:177] Using model weights format ['*.safetensors']
INFO 04-10 18:22:17 model_runner.py:104] Loading model weights took 6.8127 GB
INFO 04-10 18:22:22 gpu_executor.py

## Load the movies dataset as a DuckDB table

In [ ]:
# Make sure you import duckdb from llmsql
from llmsql.duckdb import duckdb

# Create a table from the movies dataset
conn = duckdb.connect(database=':memory:', read_only=False)
conn.execute("CREATE TABLE movies AS SELECT * FROM read_csv('movies_small.csv')")

In [ ]:
# View the table and fields in the table

print(conn.sql("SHOW TABLES"))

print(conn.sql("DESCRIBE movies"))

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ movies  │
└─────────┘

┌──────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │ column_type │  null   │   key   │ default │  extra  │
│       varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ rotten_tomatoes_link │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ review_content       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ movie_title          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ movie_info           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ id                   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
└──────────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘



## Run the LLM Queries

In [ ]:
query = "SELECT LLM('Recommend movies for the user based on {movie_info} and {review_content}', movie_info, review_content) FROM movies LIMIT 1"

In [ ]:
query_result = conn.execute(query).fetchall()

InvalidInputException: Invalid Input Error: Python exception occurred while executing the UDF: RuntimeError: CUDA out of memory. Tried to allocate 270.00 MiB. GPU 0 has a total capacty of 14.58 GiB of which 241.56 MiB is free. Including non-PyTorch memory, this process has 14.33 GiB memory in use. Of the allocated memory 12.21 GiB is allocated by PyTorch, and 677.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
Exception raised from malloc at ../c10/cuda/CUDACachingAllocator.cpp:1438 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x70c9040ba617 in /home/ray/anaconda3/lib/python3.9/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x30f6c (0x70c904158f6c in /home/ray/anaconda3/lib/python3.9/site-packages/torch/lib/libc10_cuda.so)
frame #2: <unknown function> + 